In [1]:
import ee
import geemap 
import geopandas
import h5py
import numpy as np
import rasterio
import pandas as pd
import glob
import os

In [2]:
ee.Authenticate(force=True)
ee.Initialize()


Successfully saved authorization token.


In [3]:
# Read csv file and get the information of sites' location (lat, lon)
sites_info = pd.read_csv('site_info.csv')
sites_info['name'] = sites_info['network']+ '_' + sites_info['station']
# print(sites_info.head())

# Get the geometry of sites
sites = {}
for i in range(len(sites_info)):
    site = sites_info.iloc[i]
    sites[site['name']] = ee.Geometry.Point(site['lon'], site['lat'])



In [4]:
dir = 'daily_ave'
file_names = os.listdir(dir)
file_names = [f.split('.c')[0] for f in file_names]
file_names

['SCAN_Bosque_Seco',
 'SCAN_Combate',
 'SCAN_Isabela',
 'SCAN_Kainaliu',
 'SCAN_Kemole_Gulch',
 'SCAN_Kukuihaele',
 'SCAN_Mana_House',
 'SCAN_Maricao_Forest',
 'SCAN_Silver_Sword',
 'SCAN_Waimea_Plain',
 'TAHMO_CRIG_(Soil_Moisture_Station_1)',
 'TAHMO_CRIG_(Soil_Moisture_Station_2)',
 'TAHMO_CSIR-SARI,_Nyankpala_-_Tamale',
 'TAHMO_KNUST_Farm,_Kumasi']

In [ ]:
dir = 'daily_ave'
file_names = os.listdir(dir)
file_names = [f.split('.c')[0] for f in file_names]
output_dir = 'daily_smap'

for file_name in file_names:
    file_path = f'{dir}/{file_name}.csv'
    print(file_path)
    # Read csv file of each site 
    df = pd.read_csv(file_path)
    print(len(df))

    # Drop rows where 'sm' columns doesn't have values
    valid_rows = df.dropna(subset=['sm'])

    point_fc = ee.FeatureCollection([ee.Feature(sites[file_name], {'name': file_name})])
    smap = ee.ImageCollection('NASA/SMAP/SPL3SMP_E/005') \
            .filterDate('2020-01-01', '2022-12-31') \
            .select(['soil_moisture_a', 'soil_moisture_pm'])
    # function get SMAP data 
    def get_smap_data(date, point_fc, smap):
        # Get SMAP image colelction
        # Load SMAP dataset
        
        smap_image = smap.filterDate(date).first()
        if smap_image:
            sampled = smap_image.sampleRegions(
            collection=point_fc, 
            scale=9000, 
            properties=['name'], 
            geometries=True  # This will keep the pixel locations
            )
            try:
                return sampled.getInfo()
            except:
                return None
        return None 
        

    time_info = valid_rows['time']

    df['smap_soil_moisture_am'] = np.nan
    df['smap_soil_moisture_pm'] = np.nan
    df['smap_lon'] = np.nan
    df['smap_lat'] = np.nan


    for index, row in valid_rows.iterrows():
        date = row['time']
        # print(index)
        smap_data = get_smap_data(date, point_fc, smap)
        if smap_data:
            if len(smap_data['features']) == 0:
                continue
            df.loc[index, 'smap_soil_moisture_am'] = 0.25
            df.loc[index, 'smap_soil_moisture_am'] = smap_data['features'][0]['properties']['soil_moisture_am']
            df.loc[index, 'smap_soil_moisture_pm'] = smap_data['features'][0]['properties']['soil_moisture_pm']
            df.loc[index, 'smap_lon'] = smap_data['features'][0]['geometry']['coordinates'][0]
            df.loc[index, 'smap_lat'] = smap_data['features'][0]['geometry']['coordinates'][1]
        else:
            continue

    df.to_csv(f'{output_dir}/{file_name}_smap.csv', index=False)
    print("Processed : ", file_name)

daily_ave/SCAN_Bosque_Seco.csv
1096
Processed :  SCAN_Bosque_Seco
daily_ave/SCAN_Combate.csv
1096
Processed :  SCAN_Combate
daily_ave/SCAN_Isabela.csv
1096
Processed :  SCAN_Isabela
daily_ave/SCAN_Kainaliu.csv
1096
Processed :  SCAN_Kainaliu
daily_ave/SCAN_Kemole_Gulch.csv
1096
Processed :  SCAN_Kemole_Gulch
daily_ave/SCAN_Kukuihaele.csv
1096
Processed :  SCAN_Kukuihaele
daily_ave/SCAN_Mana_House.csv
1096
Processed :  SCAN_Mana_House
daily_ave/SCAN_Maricao_Forest.csv
1096
Processed :  SCAN_Maricao_Forest
daily_ave/SCAN_Silver_Sword.csv
1096
Processed :  SCAN_Silver_Sword
daily_ave/SCAN_Waimea_Plain.csv
1096
Processed :  SCAN_Waimea_Plain
daily_ave/TAHMO_CRIG_(Soil_Moisture_Station_1).csv
1096
Processed :  TAHMO_CRIG_(Soil_Moisture_Station_1)
daily_ave/TAHMO_CRIG_(Soil_Moisture_Station_2).csv
1096
Processed :  TAHMO_CRIG_(Soil_Moisture_Station_2)
daily_ave/TAHMO_CSIR-SARI,_Nyankpala_-_Tamale.csv
1096
Processed :  TAHMO_CSIR-SARI,_Nyankpala_-_Tamale
daily_ave/TAHMO_KNUST_Farm,_Kumasi.csv

In [61]:
import ee

# Initialize GEE
ee.Initialize()

# Define points
points = [
    {'name': 'Hpayargyi', 'lon': 96.5302, 'lat': 17.4661},
    # {'name': 'Thanatpin', 'lon': 96.5731, 'lat': 17.2921},
    # {'name': 'Alaigni', 'lon': 96.3465, 'lat': 17.263},
    # {'name': 'Irrigation_Technology_Centre', 'lon': 96.4519, 'lat': 17.3144}
]

# Convert points to FeatureCollection
features = [ee.Feature(ee.Geometry.Point(p['lon'], p['lat']), {'name': p['name']}) for p in points]
point_fc = ee.FeatureCollection(features)

# Load SMAP dataset
smap = ee.ImageCollection('NASA/SMAP/SPL3SMP_E/005') \
        .filterDate('2020-01-01', '2022-12-31') \
        .select(['soil_moisture_am', 'soil_moisture_pm'])

# Function to sample SMAP at points for a given date
def sample_smap(date):
    # smap_image = smap.filterDate(date, ee.Date(date).advance(5, 'day')).mosaic()
    smap_image = smap.filterDate(date).first()
    if smap_image:
        sampled = smap_image.sampleRegions(
        collection=point_fc, 
        scale=9000, 
        properties=['name'], 
        geometries=True  # This will keep the pixel locations
    )
        return sampled
    return None

# Example: Extract data for a single date
sampled_data = sample_smap('2020-02-16')
if sampled_data:
    print(sampled_data.getInfo())

    print("\nCoordinates of the features:")
    for feature in sampled_data.getInfo()['features']:
        print(feature['geometry']['coordinates'])


{'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['soil_moisture_am', 'soil_moisture_pm']}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [96.54186868992244, 17.49568409019807]}, 'id': '0_0', 'properties': {'name': 'Hpayargyi', 'soil_moisture_am': 0.27425259351730347, 'soil_moisture_pm': 0.28482693433761597}}]}

Coordinates of the features:
[96.54186868992244, 17.49568409019807]


In [18]:
s = sampled_data.getInfo()

s['features'][0]['properties']['soil_moisture_am']

0.288127601146698

In [ ]:
def calculate_bounding_box(lon, lat, pixel_size):
    half_size = pixel_size / 2
    return {
        'min_lon': lon - half_size,
        'max_lon': lon + half_size,
        'min_lat': lat - half_size,
        'max_lat': lat + half_size
    }

In [ ]:
import ee
ee.Initialize()

mosaic_days = 5

# Load SMAP data 
smap = ee.ImageCollection('NASA/SMAP/SPL3SMP_E/005') \
        .filterDate('2022-12-10', '2022-12-31') \
        .select(['soil_moisture_am', 'soil_moisture_pm'])

# Define points 
points = [
    {'name': 'Hpayargyi', 'lon': 96.5302, 'lat': 17.4661},
    {'name': 'Thanatpin', 'lon': 96.5731, 'lat': 17.2921},
    {'name': 'Alaigni', 'lon': 96.3465, 'lat': 17.263},
    {'name': 'Irrigation_Technology_Centre', 'lon': 96.4519, 'lat': 17.3144}
]

# Convert points to ee.FeatureCollection
features = [ee.Feature(ee.Geometry.Point(p['lon'], p['lat']), {'name': p['name']}) for p in points]
point_fc = ee.FeatureCollection(features)

def get_smap_grid(start_date):
    end_date = ee.Date(start_date).advance(mosaic_days, 'day')
    
    # Get a mosaic image 
    mosaic = smap.filterDate(start_date, end_date).mosaic()
    
    # Find valid SMAP pixels where either AM or PM soil moisture > 0
    valid_mask = mosaic.select(["soil_moisture_am", "soil_moisture_pm"]).gt(0).reduce("max")
    
    # Define the processing region as the bounds of the points
    processing_region = point_fc.geometry().bounds()

    # Convert valid pixels to grid cells (SMAP pixels are 9 km x 9 km)
    smap_grid = valid_mask.updateMask(valid_mask).reduceToVectors(
        reducer=ee.Reducer.countEvery(),
        geometryType="centroid",  # Get grid cell centers
        scale=9000,  # SMAP resolution
        geometry=processing_region,  # Restrict operation to relevant area
        maxPixels=1e8
    )

    # Filter SMAP grid cells to only those containing the input points
    valid_grids = smap_grid.filterBounds(point_fc)

    return valid_grids

# Get the valid SMAP grid cells over the specified period
valid_grids = get_smap_grid('2022-12-20')

# Use the SMAP grid cell locations to filter the SMAP dataset before exporting
filtered_smap = smap.filterBounds(valid_grids.geometry())

# Convert filtered images to a list
filtered_images = filtered_smap.toList(filtered_smap.size())

print(filtered_smap.size().getInfo())

# Export each grid cell separately
# for i in range(filtered_images.size().getInfo()):
#     img = ee.Image(filtered_images.get(i))
#     img_date = img.date().format("YYYY-MM-dd").getInfo()
    
#     # Extract grid cell features
#     grid_list = valid_grids.toList(valid_grids.size())
#     for j in range(grid_list.size().getInfo()):
#         grid = ee.Feature(grid_list.get(j))
#         grid_geom = grid.geometry()
        
#         # Find points inside this grid cell
#         covered_points = point_fc.filterBounds(grid_geom)
#         point_names = covered_points.aggregate_array("name").getInfo()

#         if not point_names:
#             point_names = ["Unknown"]  # Fallback if no named points

#         # Format point names in filename
#         point_names_str = "_".join(point_names).replace(" ", "_")

#         # Clip SMAP image to this grid cell
#         clipped_img = img.clip(grid_geom)

#         # Export
#         task = ee.batch.Export.image.toDrive(
#             image=clipped_img,
#             description=f"SMAP_{img_date}_{point_names_str}",
#             fileNamePrefix=f"smap_{img_date}_{point_names_str}",
#             region=grid_geom.bounds().getInfo()["coordinates"],
#             scale=9000,
#             fileFormat="GeoTIFF"
#         )
        # task.start()


In [ ]:
for i in range(available_images.size().getInfo()):
    img = ee.Image(available_images.get(i))
    date = img.date().format("YYYY-MM-dd").getInfo()

    print(date)
    filtered_img = filter_smap_by_region(img)

    if filtered_img:
        print(f"Downloading SMAP grid cells for {date}")

        task = ee.batch.Export.image.toDrive(
            image=filtered_img,
            description=f"SMAP_Grid_{date}",
            fileNamePrefix=f"smap_grid_{date}",
            region=filtered_img.geometry().bounds().getInfo()["coordinates"],
            scale=9000,
            fileFormat="GeoTIFF"
        )

In [20]:
import utils
s_time="2020-01-01" # start and end date
e_time="2022-12-31"
ismn_data = pd.read_csv('SMN_raw/VDS/Hpayargyi/VDS_VDS_Hpayargyi_sm_0.100000_0.100000_GS1-Port-2_20200101_20221231.stm', comment='#', delim_whitespace=True)
path = 'SMN_raw/VDS/Hpayargyi/VDS_VDS_Hpayargyi_sm_0.100000_0.100000_GS1-Port-2_20200101_20221231.stm'
# ismn_data = utils.readstm_all(path,'sm',s_time, e_time)

/tmp/ipykernel_5099/537135825.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ismn_data = pd.read_csv('SMN_raw/VDS/Hpayargyi/VDS_VDS_Hpayargyi_sm_0.100000_0.100000_GS1-Port-2_20200101_20221231.stm', comment='#', delim_whitespace=True)
